In [2]:
import deap as dp
import random
import numpy as np

class ACO:
    def __init__(self, cidades, distancias, tamanho_populacao, numero_geracoes, taxa_feromonio, taxa_evaporacao):
        self.cidades = cidades
        self.distancias = distancias
        self.tamanho_populacao = tamanho_populacao
        self.numero_geracoes = numero_geracoes
        self.taxa_feromonio = taxa_feromonio
        self.taxa_evaporacao = taxa_evaporacao

        self.toolbox = dp.tools.setup(lambda n: dp.Individual(np.random.permutation(cidades)))
        self.toolbox.register("evaluate", self.fitness)

        self.populacao = self.toolbox.population(n=self.tamanho_populacao)

    def fitness(self, rota):
        tempo_viagem = 0
        distancia_percorrida = 0
        custo_transporte = 0

        for i in range(len(rota) - 1):
            cidade_atual = rota[i]
            proxima_cidade = rota[i + 1]

            tempo_viagem += self.distancias[(cidade_atual, proxima_cidade)]
            distancia_percorrida += self.distancias[(cidade_atual, proxima_cidade)]
            custo_transporte += self.distancias[(cidade_atual, proxima_cidade)] * 2

        return tempo_viagem, distancia_percorrida, custo_transporte

    def calcular_probabilidade_transicao(self):
        probabilidades_transicao = {}
        for cidade_atual in self.cidades:
            for proxima_cidade in self.cidades:
                if cidade_atual!= proxima_cidade:
                    feromonio = 0
                    for rota in self.populacao:
                        if cidade_atual in rota and proxima_cidade in rota:
                            feromonio += 1 / self.fitness(rota)[0]

                    probabilidades_transicao[(cidade_atual, proxima_cidade)] = feromonio

        return probabilidades_transicao

    def gerar_novas_rotas(self, probabilidades_transicao):
        nova_populacao = []
        for individuo in self.populacao:
            nova_rota = []
            cidade_atual = individuo[0]
            for i in range(len(self.cidades) - 1):
                proxima_cidade = random.choice(self.cidades)
                probabilidade_acumulada = 0
                for possivel_proxima_cidade in self.cidades:
                    if possivel_proxima_cidade!= cidade_atual:
                        probabilidade_acumulada += probabilidades_transicao[(cidade_atual, possivel_proxima_cidade)]
                        if random.random() < probabilidade_acumulada:
                            proxima_cidade = possivel_proxima_cidade
                            break

                nova_rota.append(proxima_cidade)
                cidade_atual = proxima_cidade

            nova_populacao.append(nova_rota)

        return nova_populacao

    def depositar_feromonio(self, populacao, probabilidades_transicao):
        for rota in populacao:
            for i in range(len(rota) - 1):
                cidade_atual = rota[i]
                proxima_cidade = rota[i + 1]
                feromonio = 1 / self.fitness(rota)[0]
                distancia = self.distancias[(cidade_atual, proxima_cidade)]

                feromonio_antigo = feromonio / probabilidades_transicao[(cidade_atual, proxima_cidade)]
                probabilidades_transicao[(cidade_atual, proxima_cidade)] = feromonio

        return probabilidades_transicao

    def evoluir(self):
        for geracao in range(self.numero_geracoes):
            probabilidades_transicao = self.calcular_probabilidade_transicao()
            nova_populacao = self.gerar_novas_rotas(probabilidades_transicao)
            self.populacao = nova_populacao
            probabilidades_transicao = self.depositar_feromonio(self.populacao, probabilidades_transicao)

if __name__ == "__main__":
    cidades = ["A", "B", "C", "D", "E"]
    distancias = {
        ("A", "B"): 10,
        ("A", "C"): 15,
        ("A", "D"): 20,
        ("A", "E"): 25,
        ("B", "C"): 12,
        ("B", "D"): 18,
        ("B", "E"): 23,
        ("C", "D"): 11,
        ("C", "E"): 16
    }

    print("ALG EVOLUTVO FORMIGAS")
    aco = ACO(cidades, distancias, 10, 100, 0.1, 0.1)
    aco.evoluir()
    print("alg evolutivo")

ALG EVOLUTVO FORMIGAS


AttributeError: module 'deap' has no attribute 'tools'

In [ ]:
import seaborn as sns


def plot_fitness_distribution(all_individuals_data, title="Distribuição do Fitness"):
    """
    Gera um boxplot mostrando a distribuição do fitness por geração.

    Args:
        all_individuals_data: Lista de dicionários contendo informações dos indivíduos (geração, fitness).
        title: Título do gráfico (opcional).
    """
    generations = []
    fitnesses = []
    for individual in all_individuals_data:
        generations.append(individual["Generations"])
        fitnesses.append(individual["Fitness"][0])

    df = pd.DataFrame({"Geração": generations, "Fitness": fitnesses})
    sns.boxplot(x="Geração", y="Fitness", showmeans=True, data=df)
    plt.title(title)
    plt.xlabel("Geração")
    plt.ylabel("Fitness")
    plt.grid(True)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt


def matrizTempo():

    # Criando a matriz de tempos
    matriz_tempos = np.zeros((num_tarefas, num_tarefas))

    for i in range(num_tarefas):
        for j in range(num_tarefas):
            if i == j:
                matriz_tempos[i, j] = tempos_tarefas[i]
            else:
                sequencia_parcial = permutacao_otima[:j + 1]
                matriz_tempos[i, j] = tempo_total(sequencia_parcial)

    # Visualizando a matriz de tempos com Seaborn
    plt.figure(figsize=(10, 6))
    heatmap(matriz_tempos, annot=True, fmt="d")
    plt.xlabel("Posição na Sequencia")
    plt.ylabel("Tarefa")
    plt.title("Matriz de Tempos")
    plt.show()

def matrizCorrelacao():
    pass




def plot_diversity_fitness(all_individuals_data, title="Diversidade vs. Fitness"):
    """
    Gera um gráfico de dispersão mostrando a diversidade vs. fitness dos indivíduos.

    Args:
        all_individuals_data: Lista de dicionários contendo informações dos indivíduos (geração, diversidade, fitness).
        title: Título do gráfico (opcional).
    """
    generations = []
    diversities = []
    fitnesses = []
    for individual in all_individuals_data:
        generations.append(individual["Generations"])
        diversities.append(sum(individual["Variaveis de Decisão"]))

    plt.figure(figsize=(10, 6))
    plt.scatter(diversities, fitnesses, c=generations, cmap="viridis")
    plt.colorbar(label="Geração")
    plt.xlabel("Diversidade")
    plt.title(title)
    plt.grid(True)
    plt.show()